In [ ]:
!pip install flask-ngrok openai-whisper transformers torch

In [ ]:
from flask import Flask, request, jsonify, render_template_string
import whisper
from transformers import pipeline
from pyngrok import ngrok
import torch

# Load models
asr_model = whisper.load_model("small")   # speech-to-text
summary_model = pipeline("summarization", model="facebook/bart-large-cnn")

app = Flask(__name__)

Device set to use cpu


In [ ]:
ngrok.set_auth_token("343bhED98AgKpTGW2LdZW2fPi98_3ZUC6BTL6oiwRUVypXvbB")

In [ ]:
HTML_PAGE = """
<!DOCTYPE html>
<html>
<head>
  <title>🎓 Lecture Voice-to-Notes</title>
  <style>
    body {
      background: linear-gradient(135deg, #74ABE2, #5563DE);
      font-family: 'Segoe UI', Tahoma, sans-serif;
      color: white;
      text-align: center;
      padding: 40px;
    }
    .container {
      background: rgba(255,255,255,0.15);
      backdrop-filter: blur(10px);
      border-radius: 20px;
      padding: 30px;
      max-width: 600px;
      margin: auto;
      box-shadow: 0 4px 20px rgba(0,0,0,0.2);
    }
    input[type=file] {
      margin-top: 15px;
      background: white;
      border-radius: 10px;
      padding: 10px;
      color: black;
    }
    button {
      background: #00C2CB;
      color: white;
      border: none;
      border-radius: 10px;
      padding: 10px 20px;
      font-size: 16px;
      margin: 10px;
      cursor: pointer;
      transition: background 0.3s;
    }
    button:hover { background: #0099A8; }
    h2 { margin-bottom: 20px; }
    h3 { color: #FFD700; }
    audio { margin-top: 20px; }
  </style>
</head>
<body>
    <h1>🎓 Lecture Voice-to-Notes Generator</h1>
  <div class="container">


    <!-- 🎙️ Recording Controls -->
    <button id="recordBtn">🎙️ Start Recording</button>
    <button id="stopBtn" disabled>⏹️ Stop</button>
    <audio id="audioPlayback" controls style="display:none;"></audio>

    <form id="uploadForm" action="/upload" method="post" enctype="multipart/form-data">
        <input id="audioInput" type="file" name="audio" accept="audio/*" required><br>
        <button type="submit">📤 Upload & Convert</button>
    </form>

    {% if text %}
    <h3>🗣 Transcribed Text:</h3>
    <p>{{ text }}</p>
    <h3>🧾 Summarized Notes:</h3>
    <p>{{ summary }}</p>
    {% endif %}
  </div>

  <script>
    let mediaRecorder, audioChunks = [];
    const recordBtn = document.getElementById("recordBtn");
    const stopBtn = document.getElementById("stopBtn");
    const audioInput = document.getElementById("audioInput");
    const audioPlayback = document.getElementById("audioPlayback");

    recordBtn.onclick = async () => {
      const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
      mediaRecorder = new MediaRecorder(stream);
      mediaRecorder.start();
      recordBtn.disabled = true;
      stopBtn.disabled = false;
      audioChunks = [];

      mediaRecorder.ondataavailable = e => { audioChunks.push(e.data); };
      mediaRecorder.onstop = e => {
        const audioBlob = new Blob(audioChunks, { type: 'audio/wav' });
        const audioUrl = URL.createObjectURL(audioBlob);
        audioPlayback.src = audioUrl;
        audioPlayback.style.display = "block";

        // Put blob into file input for upload
        const file = new File([audioBlob], "recorded.wav", { type: "audio/wav" });
        const dt = new DataTransfer();
        dt.items.add(file);
        audioInput.files = dt.files;
      };
    };

    stopBtn.onclick = () => {
      mediaRecorder.stop();
      recordBtn.disabled = false;
      stopBtn.disabled = true;
    };
  </script>
</body>
</html>
"""


@app.route('/')
def index():
    return render_template_string(HTML_PAGE)

@app.route('/upload', methods=['GET','POST'])
def upload_audio():
    audio_file = request.files['audio']
    path = "uploaded.wav"
    audio_file.save(path)

    result = asr_model.transcribe(path)
    text = result['text']

    summary = summary_model(text, max_length=100, min_length=30, do_sample=False)[0]['summary_text']

    return render_template_string(HTML_PAGE, text=text, summary=summary)


In [ ]:
public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)
app.run()

🌐 Public URL: NgrokTunnel: "https://sapotaceous-melissa-unimpropriated.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
